# Калибровка

## Импорт необходимых модулей/функций

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random
from tqdm.notebook import trange

from pixelpoint.calibration import load_images, save_calibration_params, calibrate_camera_chessboard, stereo_calibrate_chessboard, calculate_CM, calculate_RT

## Константы

_cam1_ - левая камера  
_cam2_ - правая камера

In [ ]:
pattern_params = {
    'square_size': (0.01, 0.01),  # size of chess square (in m)
    'chessboard_size': (7, 7)
}
camera_params = {
    'focal_length_m': 80e-3,
    'img_resolution': (5120, 4096),
    'pixel_sz_x': 4.5e-6,  # in m
    'pixel_sz_y': 4.5e-6  # in m
}
stereo_params = {
    'baseline': 412e-3,  # in m
    'center_dist_to_obj': 639e-3,  # in m
    'cam_dist_to_obj': 650e-3,  # in m
}

## Данные

Данные сгенерированы с помощью скрипта `render_chessboard.py`.  

In [ ]:
imgs_folder = Path('calibration_images_chessboard')
cam1_imgs, cam2_imgs = load_images(imgs_folder)

In [ ]:
random_keys = random.sample(list(cam1_imgs.keys()), 3)

fig, axes = plt.subplots(3, 2, figsize=(10, 10))

for i, key in enumerate(random_keys):
    left_img = cam1_imgs[key]
    
    right_img_key = key.replace('cam2', 'cam1')
    right_img = cam2_imgs.get(right_img_key, None)
    
    axes[i, 0].imshow(left_img, cmap='gray')
    axes[i, 0].set_title(f"Left: {key}")
    axes[i, 0].axis('off')
    
    if right_img is not None:
        axes[i, 1].imshow(right_img, cmap='gray')
        axes[i, 1].set_title(f"Right: {right_img_key}")
        axes[i, 1].axis('off')
    else:
        axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

## Поиск внутренних параметров камер

### Вычисление camera matrix

Camera matrix можно вычислить исходя из следующих параметров:
1. фокусное расстояние: 80mm
2. размер пикселя: 4.5 µm × 4.5 µm
3. разрешение изображения: 5120 × 4096 

In [ ]:
CM = calculate_CM(**camera_params)
print(f'Camera matrix:\n{CM}')

### Поиск всех внутренних параметров с помощью калибровки шахматной доской

In [ ]:
CM, dist = calibrate_camera_chessboard({**cam1_imgs, **cam2_imgs}, camera_params['img_resolution'], 
                              pattern_params['square_size'][0], pattern_params['chessboard_size'])
print(f'Camera matrix:\n{CM}')
print(f'Distortion vector:\n{dist}')

## Стерео-калибровка

### Manual estimation of extrinsic parameters for a given optical system configuration

In [ ]:
R, T = calculate_RT(stereo_params['baseline'], stereo_params['cam_dist_to_obj'])

print(f'Rotation matrix:\n{R}')
print(f'Translation vector:\n{T}')

### Stereo-calibration using the calibration images with chessboard

R: The relative rotation matrix between the first and second cameras. This matrix describes how the second camera is rotated relative to the first camera.

T: The translation vector between the first and second cameras. This vector describes how far and in which direction the second camera is from the first camera.

E: The essential matrix. This matrix encodes the rotation and translation between the two cameras, and it can be used to compute the epipolar geometry (constraints for stereo matching).

F: The fundamental matrix. This is another way of representing the geometric relationship between the two cameras. It defines the epipolar lines in stereo matching.

In [ ]:
R, T, E, F = stereo_calibrate_chessboard(cam1_imgs, cam2_imgs, camera_params['img_resolution'], 
                                pattern_params['square_size'][0], pattern_params['chessboard_size'], 
                                CM, dist)

print(f'Rotation matrix:\n{R}')
print(f'Translation vector:\n{T}')

Проверим baseline и углы поворота для полученных матриц

In [ ]:
def rotation_matrix_to_euler_angles(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    
    singular = sy < 1e-6
    
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0

    return np.degrees(np.array([yaw, pitch, roll]))

def show_baseline_and_rotation(T_calculated, R_calculated, real_baseline, R_real=None):
    calculated_baseline = np.linalg.norm(T)
    
    print(f"Calculated baseline: {calculated_baseline:.2f} m")
    print(f"Real baseline: {real_baseline:.2f} m")
    
    baseline_difference = np.abs(calculated_baseline - real_baseline)
    print(f"Baseline difference: {baseline_difference:.2f} m")
    
    euler_angles_calculated = rotation_matrix_to_euler_angles(R_calculated)
    print("\nCalculated Euler angles (Yaw, Pitch, Roll):")
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_baseline_and_rotation(T, R, stereo_params['baseline'])

## Export the camera's intrinsic and extrinsic parameters

In [ ]:
save_calibration_params(CM, dist, R, T, E, F, 'calibration_params/calibration_synthetic.json')